# Running Ollama with VDS Tunnel in Google Colab

## Настройка

1. В Colab добавьте секреты (значок ключа слева):
   - vds_host: IP вашего VDS
   - vds_user: имя пользователя
   - vds_port: SSH порт
   - ssh_key: приватный SSH-ключ в base64 (конвертируйте командой `base64 -w0 ~/.ssh/id_rsa`)

2. На VDS:
   - Добавьте публичный ключ в `~/.ssh/authorized_keys`
   - Откройте порт 11434
   - Разрешите создание обратного туннеля в `/etc/ssh/sshd_config`: `GatewayPorts yes`

3. Используйте GPU runtime (Runtime -> Change runtime type -> GPU)

## 1. Установка пакетов

In [ ]:
# Install Ollama
!curl https://ollama.ai/install.sh | sh

# Install CUDA and SSH
!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers openssh-client

## 2. Настройка окружения и импорт зависимостей

In [ ]:
from google.colab import userdata
import os
import subprocess
import threading
import time
import base64

# Проверка наличия всех необходимых секретов
required_secrets = ['vds_host', 'vds_user', 'vds_port', 'ssh_key']
missing_secrets = [secret for secret in required_secrets if not userdata.get(secret)]
if missing_secrets:
    raise ValueError(f"Missing required secrets: {', '.join(missing_secrets)}\n"
                    "Add them in Colab: Left sidebar -> Key icon -> Add new secret\n"
                    "Required secrets:\n"
                    "- vds_host: IP адрес VDS\n"
                    "- vds_user: Имя пользователя\n"
                    "- vds_port: SSH порт (обычно 22)\n"
                    "- ssh_key: SSH приватный ключ в base64")

# Создаем директорию .ssh и сохраняем ключ
ssh_dir = os.path.expanduser('~/.ssh')
os.makedirs(ssh_dir, mode=0o700, exist_ok=True)

# Декодируем и сохраняем SSH ключ
key_data = base64.b64decode(userdata.get('ssh_key')).decode()
key_path = os.path.join(ssh_dir, 'id_rsa')
with open(key_path, 'w') as f:
    f.write(key_data)
os.chmod(key_path, 0o600)

# Конфигурация из секретов
VDS_HOST = userdata.get('vds_host')
VDS_USER = userdata.get('vds_user')
VDS_PORT = userdata.get('vds_port')
LOCAL_PORT = 11434

# Set LD_LIBRARY_PATH for NVIDIA
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

## 3. Вспомогательные функции

In [ ]:
def run_and_print_output(process):
    """Helper function to continuously read and print process output"""
    while True:
        output = process.stdout.readline()
        if output == '' and process.poll() is not None:
            break
        if output:
            print(output.strip())

## 4. Запуск SSH туннеля и Ollama

In [ ]:
# Создание SSH туннеля
ssh_command = f"ssh -N -R {LOCAL_PORT}:localhost:{LOCAL_PORT} -i {key_path} {VDS_USER}@{VDS_HOST} -p {VDS_PORT}"
ssh_process = subprocess.Popen(
    ssh_command.split(),
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

# Запуск Ollama
print('>>> starting ollama serve')
ollama_serve = subprocess.Popen(
    ['ollama', 'serve'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

serve_thread = threading.Thread(target=run_and_print_output, args=(ollama_serve,))
serve_thread.daemon = True
serve_thread.start()

time.sleep(5)

## 5. Загрузка модели

In [ ]:
# Загрузка модели
print('>>> starting ollama pull qwen2.5-coder:32b')
# ***************************************************
# Измените название модели на нужную вам
# Смотрите библиотеку моделей на сайте Ollama
# ***************************************************
pull_process = subprocess.Popen(
    ['ollama', 'pull', 'qwen2.5-coder:32b'],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True
)

while True:
    output = pull_process.stdout.readline()
    if output == '' and pull_process.poll() is not None:
        break
    if output:
        print(output.strip())

## 6. Показать URL и поддерживать работу сервера

In [ ]:
print(f"\n=== Your Ollama server is available at ===")
print(f"http://{VDS_HOST}:{LOCAL_PORT}")
print("=====================================")

try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("Shutting down...")
    ollama_serve.terminate()
    ssh_process.terminate()